In [1]:
# read in data, label with isolation forest, and filter the anomalies.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans

mon = pd.read_csv('~/Desktop/data/netFlow-02-14-2017.csv', sep = ',')


In [11]:
mon_collab = mon[(((mon.srcAddr == '128.143.2.75') & (mon.srcPort == 443)) | ((mon.dstAddr =='128.143.2.75') & (mon.dstPort == 443)))]
mon_collab_clean = mon_collab[["bytes","pkts","dur","rate"]]
len(mon_collab_clean)

484501

In [2]:
# # instead of port 443 we look at port 80
# mon_collab = mon[(((mon.srcAddr == '128.143.2.75') & (mon.srcPort == 80)) | ((mon.dstAddr =='128.143.2.75') & (mon.dstPort == 80)))]
# mon_collab_clean = mon_collab[["bytes","pkts","dur","rate"]]

In [12]:
len(mon_collab_clean)

484501

In [13]:
clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
clf.fit(mon_collab_clean)
y_pred = clf.predict(mon_collab_clean)
mon_collab_clean['ISO_label'] = y_pred
anomalies = mon_collab_clean[mon_collab_clean.ISO_label == -1]
len(anomalies)

/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


24226

In [4]:
clf = IsolationForest(max_samples=1000, contamination=0.05,random_state= 5, bootstrap=False)
clf.fit(mon_collab_clean)
y_pred = clf.predict(mon_collab_clean)
mon_collab_clean['ISO_label'] = y_pred

/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/sklearn/ensemble/iforest.py:183: UserWarning: max_samples (1000) is greater than the total number of samples (255). max_samples will be set to n_samples for estimation.
  % (self.max_samples, n_samples))
/Users/ChineseSamurai/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
mon_collab_clean.head()

,bytes,pkts,dur,rate,ISO_label
21671,2628,30,0.386,54466.33,1
54799,2628,30,0.615,34185.37,1
74985,2400,30,0.237,81012.66,1
173300,1392,24,11.029,1009.70,1
180020,2628,30,0.358,58726.25,1


In [6]:
anomalies = mon_collab_clean[mon_collab_clean.ISO_label == -1]

In [7]:
len(anomalies)

13

In [8]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.spatial import distance
## function adopted from paper, KSE
def euc_dis(p1,p2):
    a = p1.values
    b = p2.values
    c = np.vstack((a,b))
    dist = distance.pdist(c)
    return float(dist)
def KSE_Test(df, sample_size):
    # initiate two sample dfs, one distance matrix, and one kse score.
    # generate 2 random samples.
    sample1 = df.sample(n = sample_size, replace = False)
    sample2 = df.sample(n = sample_size, replace = False)
    kse_score = [None] * len(df) # empty list to store KSE scores
#   distance_matrix = np.matrix([[None] * int(len(df) * fraction) , 
#                               [None] * int(len(df) * fraction)])
#     distance_matrix = [[0 for x in range(len(sample1))] 
#                        for y in range(len(sample2))] 
    distance_matrix = np.zeros((sample_size, sample_size))
    # calculate distance from all points in sample1 to all points in sample2
    for i in range(0,sample_size):
        for j in range(0,sample_size):
            distance_matrix[i][j] = euc_dis(sample1.iloc[i,],sample2.iloc[j,])
    print("half way there")
    
    # calculate average KS-score for each instance in df.
    for i in range(0, len(df)):
        observed_instance = df.iloc[i,]
        distance_array = [0] * sample_size
        for j in range(0,sample_size):
            distance_array[j] = euc_dis(observed_instance,sample2.iloc[j,])
        sum = 0
        for k in range(0, sample_size):
            ks_score = stats.ks_2samp(distance_array, distance_matrix[k])[0]
            sum += ks_score
        average = sum/sample_size
        kse_score[i] = average
        if (i%100 == 0):
            print('loop %s' % i)
    return kse_score 

In [ ]:
kse_scores = KSE_Test(anomalies, 200)

half way there
loop 0
loop 100
loop 200
loop 300
loop 400
loop 500
loop 600
loop 700
loop 800
loop 900
loop 1000
